<a href="https://colab.research.google.com/github/YG15/DataHack2019/blob/master/model_train_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sklearn as sk
import matplotlib.pyplot as plt
import json
from urllib import request
import random
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

In [3]:
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
url = 'https://armis-datahack.s3.amazonaws.com/all_devices.csv'
devices_df = pd.read_csv(url, index_col = 0)
#devices_df.head()

In [0]:
final_df = pd.read_csv(r'drive/My Drive/only_session_Features_no_log.csv')

In [0]:
final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna(0)

In [0]:
from sklearn.mixture import GaussianMixture

def run_IF(df):
    model = IsolationForest(behaviour = 'new', max_samples=100, 
                          random_state = 1, contamination = 0)
    model.fit(df)
    anomaly_scores = model.decision_function(df)
    
    return anomaly_scores

def run_GMM(df):
    model = GaussianMixture(n_components=devices_df['type'].nunique())
    model.fit(df)
    anomaly_scores = model.decision_function(df)
    
    return anomaly_scores

In [0]:
#find most varied features
variety_df = pd.DataFrame(columns=['feature', 'max','min','std'],)
for i,c in enumerate(df_gb.columns):
  df2 =pd.DataFrame({'feature':c,'max':df_gb[c].max(),'min': df_gb[c].min(),'std':df_gb[c].std()}, index=[0])
  variety_df = variety_df.append(df2, ignore_index=True)

In [0]:
variety_df.sort_values('std', ascending=False)['feature'].values

In [16]:
df_features = final_df[[ 'network_id',
                      'device_id',
                      'hostnunique',
                      'host_ipnunique',
                      'port_dstnunique',
                      'timestampnunique',
                      'timestampcount',
                      'timestamprange',
                      'transport_protocolnunique',
                      'packets_countnanmin',
                      'packets_countnanmax',
                      'packets_countnanmean',
                      'packets_countnanstd' ]].dropna(axis= 1)

X = df_features.copy()
X.drop(['device_id'], inplace=True, axis=1)

# Run model
anomaly_scores = run_GMM(X)
anomaly_scores = -anomaly_scores

# Save results
results = pd.DataFrame({'network_id' : df_features.network_id, 'device_id' : df_features.device_id,'confidence' : anomaly_scores})

AttributeError: ignored

In [17]:
model = GaussianMixture(n_components=devices_df['type'].nunique())
model.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=7, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [24]:
prob = model.predict_proba(X)
[1-max(val) for val in prob]

[6.211244273357508e-07,
 0.0,
 0.0,
 1.2037760643845985e-07,
 3.015837066955296e-05,
 3.2923988391608816e-05,
 1.9543065914784385e-05,
 7.013910900965925e-07,
 5.120170953887282e-11,
 0.45225270067102097,
 5.446215338378302e-06,
 1.654704965270426e-07,
 0.030214688878467788,
 1.1621580886522764e-07,
 3.6564529182214756e-11,
 3.851141627819743e-11,
 1.4017061178428492e-08,
 0.003279112323138711,
 0.0,
 4.263256414560601e-14,
 1.3494889745668814e-06,
 0.0,
 9.023892744153272e-13,
 4.179673546156337e-07,
 3.106441637257262e-08,
 1.0397670225481548e-05,
 9.746662732457168e-08,
 0.0,
 6.745892733306391e-11,
 6.700417998217745e-11,
 0.0,
 0.0,
 1.3442248689710112e-06,
 4.1168846109940205e-11,
 3.3651303965598345e-11,
 0.0,
 3.2160764863675695e-05,
 2.4067645654213266e-09,
 2.0941115508321673e-10,
 2.071587346108572e-10,
 5.7801605390928046e-08,
 1.6502107369475993e-06,
 3.015837066955296e-05,
 0.0,
 1.311519781665993e-10,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.4210854715202004e-14,
 3.386226

In [0]:
display(results.head())
print(results.shape)

,network_id,device_id,confidence
0,0,33,-0.429968
1,0,35,-0.266061
2,0,40,-0.191984
3,0,41,-0.429760
4,0,53,-0.434053


(76038, 3)


In [0]:
#%% Submission

arr_to_submit = results.to_json(orient='values')

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "The Whale and the Petunias_y2"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

{'member': 'The Whale and the Petunias_y2', 'rank': 15, 'score': 0.7992275174894156}


In [0]:
#log
# features (log)
feature_cols=['packets_count',
       'outbound_bytes_count', 'inbound_bytes_count', 'packet_loss',
       'retransmit_count', 'latency', 'session_count',
       'outbound_packets_count', 'inbound_packets_count', 'outbound_bytes_max',
       'outbound_bytes_min', 'outbound_bytes_mean', 'outbound_bytes_median',
       'outbound_bytes_stddev', 'inbound_bytes_max', 'inbound_bytes_min',
       'inbound_bytes_mean', 'inbound_bytes_median', 'inbound_bytes_stddev',
       'outbound_packet_size_max', 'outbound_packet_size_min',
       'outbound_packet_size_mean', 'outbound_packet_size_median',
       'outbound_packet_size_stddev', 'inbound_packet_size_max',
       'inbound_packet_size_min', 'inbound_packet_size_mean',
       'inbound_packet_size_median', 'inbound_packet_size_stddev']

general_cols=['network_id','device_id','timestamp']
df_residuals= sessions_df.merge(devices_df[['network_id','device_id','type']], on=['network_id','device_id'], how='left')
for col in feature_cols:
    df_residuals[col]=np.log10(sessions_df[col]+1)
    # add column of average er type and timestamp
df_avgtime = df_residuals.groupby(['type','timestamp'],as_index=False).mean()
df_residuals2= df_residuals.merge(df_avgtime, how='left', on=['type','timestamp'],suffixes=('_left', '_right'))
#subtract average
new_columns = ['network_id','device_id']
for col in feature_cols:
    df_feature2['residual_'+col]= df_residuals2[col+'__left']-df_residuals2[col+'_right']
    new_columns.append('residual_'+col)
feature_residuals= df_residuals2[new_columns]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log10
